In [1]:
import os

In [2]:
%pwd

'c:\\Users\\Prachi\\Documents\\Chicken-Disease-Classification-Project\\research'

In [4]:
os.chdir("../")

In [5]:
%pwd

'c:\\Users\\Prachi\\Documents\\Chicken-Disease-Classification-Project'

In [9]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class PrepareBaseModelConfig:
    root_dir : Path
    base_model_path : Path
    updated_base_model_path : Path
    params_image_size : list
    params_include_top : bool
    params_weights : str
    params_learning_rate : float
    params_classes : int

In [5]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml,create_directories

In [6]:
from pathlib import Path

project_root = Path(r"C:\Users\Prachi\Documents\Chicken-Disease-Classification-Project")

CONFIG_FILE_PATH = project_root / "config" / "config.yaml"
PARAMS_FILE_PATH = project_root / "params.yaml"

print("Config file path:", CONFIG_FILE_PATH)
print("Params file path:", PARAMS_FILE_PATH)

print("Config exists?", CONFIG_FILE_PATH.is_file())
print("Params exists?", PARAMS_FILE_PATH.is_file())


Config file path: C:\Users\Prachi\Documents\Chicken-Disease-Classification-Project\config\config.yaml
Params file path: C:\Users\Prachi\Documents\Chicken-Disease-Classification-Project\params.yaml
Config exists? True
Params exists? True


In [8]:
class ConfigurationManager:
    def __init__(self,
                 config_filepath = CONFIG_FILE_PATH,
                 params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root], verbose=True)

    def get_prepare_base_model_config(self) -> PrepareBaseModelConfig:
        config=self.config.prepare_base_model

        create_directories([config.root_dir], verbose=True)

        prepare_base_model_config = PrepareBaseModelConfig(
            root_dir = Path(config.root_dir),
            base_model_path= Path(config.base_model_path),
            updated_base_model_path = Path(config.updated_base_model),
            params_image_size = self.params.IMAGE_SIZE,
            params_include_top = self.params.INCLUDE_TOP,
            params_weights = self.params.WEIGHTS,
            params_learning_rate = self.params.LEARNING_RATE,
            params_classes = self.params.CLASSES,
        )

        return prepare_base_model_config

In [51]:
from pathlib import Path
import os
import urllib.request as request
from zipfile import ZipFile
import tensorflow as tf
from cnnClassifier import logger
from cnnClassifier.utils.common import get_size


In [ ]:
from pathlib import Path
import tensorflow as tf
from cnnClassifier.entity.config_entity import PrepareBaseModelConfig  # if used

class PrepareBaseModel:
    def __init__(self, config: PrepareBaseModelConfig):
        self.config = config

        # 🔧 Ensure base_model_path and updated_base_model_path are Path objects
        self.base_model_path = Path(self.config.base_model_path)
        self.updated_base_model_path = Path(self.config.updated_base_model_path)

    def get_base_model(self):
        self.model = tf.keras.applications.vgg16.VGG16(
            input_shape=self.config.params_image_size,
            weights=self.config.params_weights,
            include_top=self.config.params_include_top
        )

        # 🔧 Ensure with_suffix works by using Path object
        keras_path = self.config.base_model_path.with_suffix(".keras")
        self.save_model(path=keras_path, model=self.model)

    @staticmethod
    def prepare_full_model(model, classes, freeze_all, freeze_till, learning_rate):
        if freeze_all:
            for layer in model.layers:
                layer.trainable = False  # 🔧 Fix: set layer.trainable not model.trainable
        elif freeze_till is not None and freeze_till > 0:
            for layer in model.layers[:-freeze_till]:
                layer.trainable = False

        flatten_in = tf.keras.layers.Flatten()(model.output)
        prediction = tf.keras.layers.Dense(classes, activation='softmax')(flatten_in)

        full_model = tf.keras.models.Model(
            inputs=model.input,
            outputs=prediction
        )

        full_model.compile(
            optimizer=tf.keras.optimizers.SGD(learning_rate=learning_rate),
            loss=tf.keras.losses.CategoricalCrossentropy(),  # 🔧 safer to use the class version
            metrics=['accuracy']
        )

        full_model.summary()
        return full_model

    def update_model(self):
        self.model = self.prepare_full_model(
            model=self.model,
            classes=self.config.params_classes,
            freeze_all=True,
            freeze_till=None,
            learning_rate=self.config.params_learning_rate
        )
        self.save_model(path=self.config.updated_base_model_path, model=self.model)

    @staticmethod
    def save_model(path: Path, model: tf.keras.Model):
        model.save(path)


In [47]:
try:
    config= ConfigurationManager()
    prepare_base_model_config = config.get_prepare_base_model_config()
    prepare_base_model = PrepareBaseModel(config=prepare_base_model_config)
    prepare_base_model.get_base_model()
    prepare_base_model.update_model()

except Exception as e:
    raise e

✅ DEBUG YAML PATH: C:\Users\Prachi\Documents\Chicken-Disease-Classification-Project\config\config.yaml
✅ DEBUG YAML CONTENT: {'artifacts_root': 'artifacts', 'data_ingestion': {'root_dir': 'artifacts/data_ingestion', 'source_url': 'https://github.com/PrachiVedant/datasets/raw/refs/heads/main/Chicken-fecal-images.zip', 'local_data_file': 'artifacts/data_ingestion/data.zip', 'unzip_dir': 'artifacts/data_ingestion'}, 'prepare_base_model': {'root_dir': 'artifacts/prepare_base_model', 'base_model_path': 'artifacts/prepare_base_model/base_model.h5', 'updated_base_model': 'artifacts/prepare_base_model/base_model_updated.h5'}} <class 'dict'>
✅ DEBUG YAML PATH: C:\Users\Prachi\Documents\Chicken-Disease-Classification-Project\params.yaml
✅ DEBUG YAML CONTENT: {'AUGMENTATION': True, 'IMAGE_SIZE': [224, 244, 3], 'BATCH_SIZE': 32, 'INCLUDE_TOP': False, 'EPOCHS': 1, 'CLASSES': 2, 'WEIGHTS': 'imagenet', 'LEARNING_RATE': 0.01} <class 'dict'>
[2025-07-05 14:40:00,447 : INFO : common : Created directory 